In [ ]:
import numpy as np
import xarray as xr

import xclimate as xclim

In [ ]:
A_FHIST = xr.open_dataset(
    "/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/" \
    "f.e21.FHIST_BGC.f19_f19_mg17.CMIP6-AMIP-2deg.001/atm/proc/tseries/month_1/" \
    "f.e21.FHIST_BGC.f19_f19_mg17.CMIP6-AMIP-2deg.001.cam.h0.AREA.200001-201412.nc",
    decode_timedelta=False)["AREA"].isel(time=0).fillna(0) / 1e6
A_FHIST.attrs["units"] = "km^2"

LND_GRID_FHIST = xclim.load_coupled_fhist_ppe("EFLX_LH_TOT", "lnd", "month_1")[["area", "landfrac"]].isel(member=0).fillna(0)
LND_GRID_FHIST = LND_GRID_FHIST.reindex_like(A_FHIST, method="nearest", tolerance=1e-3)
LA_FHIST = LND_GRID_FHIST.area * LND_GRID_FHIST.landfrac
LA_FHIST.attrs["long_name"] = "land area of grid box"

In [ ]:
PCT_LANDUNIT = xclim.load_coupled_fhist_ppe("PCT_LANDUNIT", "lnd", "month_1").reindex_like(A_FHIST, method="nearest", tolerance=1e-3)

LANDUNIT_TYPE = np.empty(9, dtype=object)
for key, item in PCT_LANDUNIT.attrs.items():
    if "ltype" in key:
        LANDUNIT_TYPE[item-1] = key

PCT_LANDUNIT = PCT_LANDUNIT.assign_coords(ltype=np.arange(1,10), landunit=("ltype", LANDUNIT_TYPE))["PCT_LANDUNIT"]

In [ ]:
AREA = xr.Dataset(
    {
        "AREA": A_FHIST,
        "LANDAREA": LA_FHIST,
        "PCT_GLC": PCT_LANDUNIT.isel(member=0, time=0, ltype=3).drop_vars(("time", "member")),
        "LANDFRAC": LND_GRID_FHIST.landfrac,
    }
).drop_vars(("time", "member"))
AREA.attrs["note"] = "CAM grid"
AREA.to_netcdf("/glade/campaign/univ/uwas0155/ppe/f.e21.FHIST_BGC.f19_f19_mg17.historical.AREA_GRID.nc")